# Multiple Linear Regression

In [25]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# read data into a DataFrame
data = pd.read_csv('http://pythontrade.com/public/Data/pro2/adv.csv',index_col=0)
data.head()
# this is the standard import if you're using "formula notation" (similar to R)
import statsmodels.formula.api as smf

## Multiple Linear Regression

Simple linear regression can easily be extended to include multiple features. This is called **multiple linear regression**:

$$y = \beta_0 + \beta_1x_1 + ... + \beta_kx_k+\epsilon$$

Each $x$ represents a different feature, and each feature has its own coefficient.



## Assumption about Linear Regression Model
- for different values of features, noises are independent and have equal variance $\sigma$
- noises are normal
- means of noises are zeroes.

These assumptions are important, without which, statistical properties of coefficient and prediction will not hold. That is, we still can get least square equation. $\hat{y} = b_0 + b_1x_1 + ... + b_kx_k$, but we cannot evaluate variance and bias of the model. 

In many dataset, these assumptions are not true at beginning, but we have technique to transform data so that assumptions are approximately hold. 


## Example

$sales = \beta_0 + \beta_1 \times TV + \beta_2 \times Radio + \beta_3 \times Newspaper+\epsilon$

Let's use Statsmodels to estimate these coefficients:

In [36]:
# create a fitted model with all three features
lm = smf.ols(formula='Sales ~ TV+Radio + Newspaper ', data=data).fit()


# print the coefficients
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Fri, 27 Nov 2015   Prob (F-statistic):           1.58e-96
Time:                        17:13:39   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      2.9389      0.312      9.422      0.000         2.324     3.554
TV             0.0458      0.001     32.809      0.000         0.043     0.049
Radio          0.1885      0.009     21.893      0.000         0.172     0.206
Newspaper     -0.0010      0.006     -0.177      0.860        -0.013     0.011
==============================================================================
Omnibus:                       60.414   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.241
Skew:                          -1.327   Prob(JB):                     1.44e-33
Kurtosis:                       6.332   Cond. No.                         454.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

$$RMSE=\sqrt{\frac{SSE}{n-k-1}}$$
where $k$ is # of features

In [53]:
lm3 = smf.ols(formula='Sales ~ TV+Radio+Newspaper ', data=data).fit()
real=data['Sales']
predict3=lm3.params[0]+lm3.params[1]*data["TV"]+lm3.params[2]*data["Radio"]+lm3.params[3]*data["Newspaper"]
SSE3=((real-predict3)**2).sum()
RMSE3=np.sqrt(SSE3/(len(real)-4))

In [52]:
lm2 = smf.ols(formula='Sales ~ TV+Radio ', data=data).fit()
real=data['Sales']
predict2=lm2.params[0]+lm2.params[1]*data["TV"]+lm2.params[2]*data["Radio"]
SSE2=((real-predict2)**2).sum()
RMSE2=np.sqrt(SSE2/(len(real)-3))

In [54]:
print "RMSE3=",RMSE3
print "RMSE2=",RMSE2

RMSE3= 1.68551037341
RMSE2= 1.68136091251


How do we interpret these coefficients? For a given amount of Radio and Newspaper ad spending, an **increase of $1000 in TV ad spending** is associated with an **increase in Sales of 45.765 widgets**.

A lot of the information we have been reviewing piece-by-piece is available in the model summary output:

What are a few key things we learn from this output?

- TV and Radio have significant **p-values**, whereas Newspaper does not. Thus we reject the null hypothesis for TV and Radio (that there is no association between those features and Sales), and fail to reject the null hypothesis for Newspaper.
- TV and Radio ad spending are both **positively associated** with Sales, whereas Newspaper ad spending is **slightly negatively associated** with Sales. (However, this is irrelevant since we have failed to reject the null hypothesis for Newspaper.)
- This model has a higher **R-squared** (0.897) than the previous model, which means that this model provides a better fit to the data than a model that only includes TV.

## Feature Selection

How do I decide **which features to include** in a linear model? Here's one idea:
- Try different models, and only keep predictors in the model if they have small p-values.
- Check whether the R-squared value goes up when you add new predictors.

What are the **drawbacks** to this approach?
- Linear models rely upon a lot of **assumptions** (such as the features being independent), and if those assumptions are violated (which they usually are), R-squared and p-values are less reliable.
- Using a p-value cutoff of 0.05 means that if you add 100 predictors to a model that are **pure noise**, 5 of them (on average) will still be counted as significant.
- R-squared is susceptible to **overfitting**, and thus there is no guarantee that a model with a high R-squared value will generalize. Below is an example:

In [23]:
# only include TV and Radio in the model
lm = smf.ols(formula='Sales ~ TV + Radio', data=data).fit()
lm.rsquared

0.89719426108289557

In [24]:
# add Newspaper to the model (which we believe has no association with Sales)
lm = smf.ols(formula='Sales ~ TV + Radio + Newspaper', data=data).fit()
lm.rsquared

0.89721063817895208

**R-squared will always increase as you add more features to the model**, even if they are unrelated to the response. Thus, selecting the model with the highest R-squared is not a reliable approach for choosing the best linear model.

There is alternative to R-squared called **adjusted R-squared** that penalizes model complexity (to control for overfitting), but it generally [under-penalizes complexity](http://scott.fortmann-roe.com/docs/MeasuringError.html).

So is there a better approach to feature selection? **Cross-validation.** It provides a more reliable estimate of out-of-sample error, and thus is a better way to choose which of your models will best **generalize** to out-of-sample data. There is extensive functionality for cross-validation in scikit-learn, including automated methods for searching different sets of parameters and different models. Importantly, cross-validation can be applied to any model, whereas the methods described above only apply to linear models.